In [ ]:

import torch
import torch.nn as nn
import os
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from torch.optim import Adam
import matplotlib.pyplot as plt

In [ ]:
def activation(name):
    if name in ['tanh', 'Tanh']:
        return nn.Tanh()
    elif name in ['relu', 'ReLU']:
        return nn.ReLU(inplace=True)
    elif name in ['lrelu', 'LReLU']:
        return nn.LeakyReLU(inplace=True)
    elif name in ['sigmoid', 'Sigmoid']:
        return nn.Sigmoid()
    elif name in ['softplus', 'Softplus']:
        return nn.Softplus(beta=4)
    elif name in ['celu', 'CeLU']:
        return nn.CELU()
    elif name in ['elu']:
        return nn.ELU()
    elif name in ['mish']:
        return nn.Mish()
    else:
        raise ValueError('Unknown activation function')


In [ ]:
class SpectralConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, modes1, modes2):
        super(SpectralConv2d, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1  
        self.modes2 = modes2

        self.scale = (1 / (in_channels * out_channels))
        self.weights1 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, dtype=torch.cfloat))
        self.weights2 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, dtype=torch.cfloat))

    # Complex multiplication
    def compl_mul2d(self, input, weights):
        return torch.einsum("bixy,ioxy->boxy", input, weights)

    def forward(self, x):
        batchsize = x.shape[0]
        # Compute Fourier coeffcients up to factor of e^(- something constant)
        x_ft = torch.fft.rfft2(x)

        # Multiply relevant Fourier modes
        out_ft = torch.zeros(batchsize, self.out_channels, x.size(-2), x.size(-1) // 2 + 1, dtype=torch.cfloat, device=x.device)
        out_ft[:, :, :self.modes1, :self.modes2] = \
            self.compl_mul2d(x_ft[:, :, :self.modes1, :self.modes2], self.weights1)
        out_ft[:, :, -self.modes1:, :self.modes2] = \
            self.compl_mul2d(x_ft[:, :, -self.modes1:, :self.modes2], self.weights2)

        # Return to physical space
        x = torch.fft.irfft2(out_ft, s=(x.size(-2), x.size(-1)))
        return x

In [ ]:
class FNO2d(nn.Module):
    def __init__(self, fno_architecture, device=None, padding_frac=1 / 4):
        super(FNO2d, self).__init__()

        self.modes1 = fno_architecture["modes"]
        self.modes2 = fno_architecture["modes"]
        self.width = fno_architecture["width"]
        self.n_layers = fno_architecture["n_layers"]
        self.retrain_fno = fno_architecture["retrain_fno"]

        torch.manual_seed(self.retrain_fno)
        # self.padding = 9 # pad the domain if input is non-periodic
        self.padding_frac = padding_frac
        self.fc0 = nn.Linear(3, self.width)  # input channel is 3: (a(x, y), x, y)

        self.conv_list = nn.ModuleList(
            [nn.Conv2d(self.width, self.width, 1) for _ in range(self.n_layers)])
        self.spectral_list = nn.ModuleList(
            [SpectralConv2d(self.width, self.width, self.modes1, self.modes2) for _ in range(self.n_layers)])

        self.fc1 = nn.Linear(self.width, 128)
        self.fc2 = nn.Linear(128, 1)

        self.to(device)

    def forward(self, x):
        x = self.fc0(x)
        x = x.permute(0, 3, 1, 2)

        x1_padding = int(round(x.shape[-1] * self.padding_frac))
        x2_padding = int(round(x.shape[-2] * self.padding_frac))
        x = F.pad(x, [0, x1_padding, 0, x2_padding])

        for k, (s, c) in enumerate(zip(self.spectral_list, self.conv_list)):

            x1 = s(x)
            x2 = c(x)
            x = x1 + x2
            if k != self.n_layers - 1:
                x = F.gelu(x)
        x = x[..., :-x1_padding, :-x2_padding]

        x = x.permute(0, 2, 3, 1)
        x = self.fc1(x)
        x = F.gelu(x)
        x = self.fc2(x)
        return x

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

In [ ]:
import scipy.io
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#conventional training method
input_tensor = torch.load('filedirectory/input_tensor.pt')
output_tensor = torch.load('filedirectory/output_tensor.pt')
print(f'Input tensor shape: {input_tensor.shape}')
print(f'Output tensor shape: {output_tensor.shape}')
dataset = TensorDataset(input_tensor, output_tensor)

# splitting data
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
training_set, testing_set = random_split(dataset, [train_size, test_size])
batch_size = 2
train_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testing_set, batch_size=batch_size, shuffle=False)

In [ ]:
input_tensor_test = torch.load('filedirectory/input_tensor_test.pt')
output_tensor_test = torch.load('filedirectory/output_tensor_test.pt')

original_size = input_tensor_test.shape[0]
print(f"Original dataset size: {original_size}")

max_values = input_tensor_test.view(original_size, -1).max(dim=1)[0]

filter_condition = max_values > 1.5
filtered_input_tensor = input_tensor_test[filter_condition]
filtered_output_tensor = output_tensor_test[filter_condition]

filtered_size = filtered_input_tensor.shape[0]

replication_factor = original_size // filtered_size  
expanded_input_tensor = filtered_input_tensor.repeat(replication_factor, 1, 1, 1)[:original_size]
expanded_output_tensor = filtered_output_tensor.repeat(replication_factor, 1,1,1)[:original_size]


In [ ]:
input_tensor_test = expanded_input_tensor
output_tensor_test = expanded_output_tensor

In [ ]:
#curriculum training
batch_size = 2
#input_tensor_test = torch.load('/content/drive/My Drive/Colab Notebooks/input_tensor_test.pt')
#output_tensor_test = torch.load('/content/drive/My Drive/Colab Notebooks/output_tensor_test.pt')

test_dataset = TensorDataset(input_tensor_test, output_tensor_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

group_ids = [1, 2, 3, 4] 
group_dataloaders = {}

for group_id in group_ids:
    input_group_file = f'filedirectory/input_group_{group_id}.pt'
    output_group_file = f'filedirectory/output_group_{group_id}.pt'

    input_group = torch.load(input_group_file)
    output_group = torch.load(output_group_file)

    group_dataset = TensorDataset(input_group, output_group)
    group_loader = DataLoader(group_dataset, batch_size=batch_size, shuffle=True)

    group_dataloaders[group_id] = group_loader

print('Grouped data loaded successfully.')

In [ ]:
fno_architecture = {
    "modes": 32,
    "width": 64,
    "n_layers": 12,
    "retrain_fno": 42
}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
fno = FNO2d(fno_architecture, device=device)

learning_rate = 0.001
epochs = 1000
step_size = 50
gamma = 0.5
epochs_per_group = 250
optimizer = Adam(fno.parameters(), lr=learning_rate, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
criterion = nn.MSELoss()


In [ ]:

n = 50
freq_print = n
num_groups = 4
model_save_dir = 'filedirectory/saved_models_step'
os.makedirs(model_save_dir, exist_ok=True)

log_save_dir = 'filedirectory/training_logs_step'
os.makedirs(log_save_dir, exist_ok=True)

train_losses = []
test_errors = []

for epoch in range(epochs):
    group_index = epoch // epochs_per_group
    if group_index >= num_groups:
        group_index = num_groups - 1
    current_group_id = group_ids[group_index]
    train_loader = group_dataloaders[current_group_id]

    fno.train()
    train_mse = 0.0
    for step, (input_batch, output_batch) in enumerate(train_loader):
        optimizer.zero_grad()

        input_batch = input_batch.to(device)
        output_batch = output_batch.to(device)

        output_pred_batch = fno(input_batch)

        loss = criterion(output_pred_batch, output_batch)
        loss.backward()
        optimizer.step()

        train_mse += loss.item()

    train_mse /= len(train_loader)
    train_losses.append(train_mse)
    scheduler.step()

    with torch.no_grad():
        fno.eval()
        test_relative_l2 = 0.0
        for step, (input_batch, output_batch) in enumerate(test_loader):
            input_batch = input_batch.to(device)
            output_batch = output_batch.to(device)

            output_pred_batch = fno(input_batch)
            loss_rel = (torch.mean((output_pred_batch - output_batch) ** 2) / torch.mean(output_batch ** 2)).sqrt() * 100
            test_relative_l2 += loss_rel.item()

        test_relative_l2 /= len(test_loader)
        test_errors.append(test_relative_l2)

    if (epoch + 1) % freq_print == 0:
        print(f"Epoch: {epoch + 1}, Train Loss: {train_mse:.6f}, Relative L2 Test Error: {test_relative_l2:.2f}%")
        model_filename = f'fno_epoch_{epoch + 1}.pth'
        model_path = os.path.join(model_save_dir, model_filename)
        torch.save(fno.state_dict(), model_path)
        print(f"Model saved to {model_path}")

train_losses_path = os.path.join(log_save_dir, 'train_losses.pt')
torch.save(train_losses, train_losses_path)

test_errors_path = os.path.join(log_save_dir, 'test_errors.pt')
torch.save(test_errors, test_errors_path)

print(f"Training losses saved to {train_losses_path}")
print(f"Test errors saved to {test_errors_path}")

Epoch: 50, Train Loss: 764.497607, Relative L2 Test Error: 44.33%
Model saved to /content/drive/My Drive/Colab Notebooks/saved_models_step/fno_epoch_50.pth
Epoch: 100, Train Loss: 723.324500, Relative L2 Test Error: 48.05%
Model saved to /content/drive/My Drive/Colab Notebooks/saved_models_step/fno_epoch_100.pth
Epoch: 150, Train Loss: 387.217780, Relative L2 Test Error: 48.10%
Model saved to /content/drive/My Drive/Colab Notebooks/saved_models_step/fno_epoch_150.pth
Epoch: 200, Train Loss: 209.936261, Relative L2 Test Error: 49.85%
Model saved to /content/drive/My Drive/Colab Notebooks/saved_models_step/fno_epoch_200.pth
Epoch: 250, Train Loss: 203.904416, Relative L2 Test Error: 50.40%
Model saved to /content/drive/My Drive/Colab Notebooks/saved_models_step/fno_epoch_250.pth
Epoch: 300, Train Loss: 212.984919, Relative L2 Test Error: 37.23%
Model saved to /content/drive/My Drive/Colab Notebooks/saved_models_step/fno_epoch_300.pth
Epoch: 350, Train Loss: 188.862432, Relative L2 Test E